<a href="https://colab.research.google.com/github/ahmadsaad007/Deep_Learning_Projects/blob/master/Fundus_Image_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from keras.applications.resnet50 import ResNet50
from keras.preprocessing import image
from keras.applications.resnet50 import preprocess_input, decode_predictions
import numpy as np
from keras.models import Sequential

Using TensorFlow backend.


In [0]:
import numpy as np
from tensorflow.keras import layers
from tensorflow import keras 
import tensorflow as tf
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import *
from keras.utils import np_utils
from keras.optimizers import Adam
from keras.metrics import categorical_crossentropy
from keras.preprocessing.image import ImageDataGenerator
from keras.layers.normalization import BatchNormalization
import itertools
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
%matplotlib inline
from keras import backend as K
from keras.applications.vgg16 import VGG16
from keras.layers import Input

In [3]:
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [12]:
train_path = "/content/drive/My Drive/Eye Disease Research/train"
test_path = "/content/drive/My Drive/Eye Disease Research/test"

imgdata = ImageDataGenerator( rescale=1.0/255.0, validation_split=0.10)
train_batches = imgdata.flow_from_directory(train_path,class_mode='categorical', subset='training', target_size=(768, 496) ,
                                            classes = ['CNV','DRUSEN','NORMAL'], batch_size = 32)
test_batches = imgdata.flow_from_directory(test_path, target_size=(768, 496) ,class_mode='categorical',
                                           classes = ['CNV','DRUSEN','NORMAL'], batch_size = 32)
valid_batches = imgdata.flow_from_directory(train_path, subset='validation', target_size=(768, 496) ,class_mode='categorical',
                                            classes = ['CNV','DRUSEN','NORMAL'], batch_size = 32)


Found 64937 images belonging to 3 classes.
Found 751 images belonging to 3 classes.
Found 7214 images belonging to 3 classes.


Lets try to see how many images do we have of each class and then use that to get the starting index of those images

In [6]:
mapper = {}
for label in train_batches.classes:
  if(label not in mapper):
    mapper[label] = 1
  else:
    val = mapper[label]
    mapper[label] = val+1
print(mapper)

{0: 33485, 1: 7764, 2: 23688}


# Let's train on a prebuilt model: VGG16:
We will need to reshape the input so that we can feed our images that are bigger than what the model was trained on.

In [7]:
vgg16_model = VGG16(input_tensor=Input(shape=(768, 496, 3)),input_shape=(768,496,3), include_top = False)
#vgg16_model.layers.pop()
vgg16_model.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 768, 496, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 768, 496, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 768, 496, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 384, 248, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 384, 248, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 384, 248, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 192, 124, 128)     0     

## Create a sequential model and add all the layers in VGG16 to it. Then add the Dense layers for your custom output:

In [8]:
model= Sequential()
model.add(vgg16_model)
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 24, 15, 512)       14714688  
Total params: 14,714,688
Trainable params: 14,714,688
Non-trainable params: 0
_________________________________________________________________


In [9]:
#Make the vgg16's layers non-trainable
for layer in model.layers:
  layer.trainable = False
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(3, activation='softmax'))

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 24, 15, 512)       14714688  
_________________________________________________________________
flatten_1 (Flatten)          (None, 184320)            0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               47186176  
_________________________________________________________________
dense_2 (Dense)              (None, 32)                8224      
_________________________________________________________________
dense_3 (Dense)              (None, 3)                 99        
Total params: 61,909,187
Trainable params: 47,194,499
Non-trainable params: 14,714,688
_________________________________________________________________


In [0]:
model.compile(Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

In [0]:
from sklearn.utils import class_weight
class_weights = class_weight.compute_class_weight('balanced', np.unique(train_batches.classes), train_batches.classes)

***The following code has some error. Review it later***

In [0]:
model.fit_generator(train_batches, steps_per_epoch=256, epochs=10,verbose=1, 
                    validation_data=valid_batches, validation_steps=32, shuffle=True, class_weight = class_weights)

Epoch 1/10
256/256 [==============================] - 2073s 8s/step - loss: 1.3659 - accuracy: 0.7709 - val_loss: 0.3229 - val_accuracy: 0.8506
Epoch 2/10
255/256 [============================>.] - ETA: 7s - loss: 0.4785 - accuracy: 0.8593 

# ***Let's train a Resnet50 model now and see how it performs on our data***

In [132]:
resnet_model = ResNet50(weights="imagenet",input_tensor=Input(shape=(768, 496, 3)),
                        input_shape=(768,496,3), include_top = False)

/usr/local/lib/python3.6/dist-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


In [128]:
resnet_model.summary()

Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_10 (InputLayer)           (None, 768, 496, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 774, 502, 3)  0           input_10[0][0]                   
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 384, 248, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 384, 248, 64) 256         conv1[0][0]                      
___________________________________________________________________________________________

In [133]:
model= Sequential()
model.add(resnet_model)
model.summary()

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Model)             (None, 24, 16, 2048)      23587712  
Total params: 23,587,712
Trainable params: 23,534,592
Non-trainable params: 53,120
_________________________________________________________________


In [134]:
for layer in model.layers:
  layer.trainable = False
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(3, activation='softmax'))

model.summary()

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Model)             (None, 24, 16, 2048)      23587712  
_________________________________________________________________
flatten_9 (Flatten)          (None, 786432)            0         
_________________________________________________________________
dense_22 (Dense)             (None, 64)                50331712  
_________________________________________________________________
dense_23 (Dense)             (None, 16)                1040      
_________________________________________________________________
dense_24 (Dense)             (None, 3)                 51        
Total params: 73,920,515
Trainable params: 50,332,803
Non-trainable params: 23,587,712
_________________________________________________________________


In [136]:
model.compile(Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
model.fit_generator(train_batches, steps_per_epoch=1024, epochs=4,verbose=1, 
                    validation_data=valid_batches, validation_steps=512, shuffle=True, class_weight = class_weights)

KeyboardInterrupt: ignored

# **Save the Model:**

In [0]:
!pip install h5py

In [0]:
model.save("/content/drive/My Drive/Eye Disease Research/model.h5")
print("Saved model to disk")

In [0]:
model_json = model.to_json()
with open("/content/drive/My Drive/Eye Disease Research/model_jsn.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model_wt.h5")
print("Saved model to disk")